In [1]:
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn import metrics
import tensorflow as tf
import pandas as pd
import numpy as np
import plotting
import hls4ml
import shap
import sys
import os

np.random.seed(77)

import matplotlib.pyplot as plt
import mplhep
plt.style.use(mplhep.style.CMS)


def print_dict(d, indent=0):
    align=20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))

/home/llr/cms/motta/.local/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/llr/cms/motta/.local/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.


In [2]:
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/settings64.sh
os.environ['PATH'] = '/data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/bin:' + os.environ['PATH']
!source /data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/settings64.sh

/bin/bash: /data/elec_soft/Xilinx_Software/Soft_2020.1/Vivado/2020.1/settings64.sh: No such file or directory
/bin/bash: /data/elec_soft/Xilinx_Software/Soft_2020.1/Vitis/2020.1/settings64.sh: No such file or directory


In [3]:
options = {
    'v'          : '16',
    'date'       : '2023_05_28',
    'inTagIdent' : '_QP',
    'inTagCalib' : '_QP',
    'caloClNxM'  : '5x9',
    'sparsity'   : 0.5
}

In [4]:
indir = '/data_CMS/cms/motta/Phase2L1T/'+options['date']+'_v'+options['v']+'/TauMinator_CB_cltw'+options['caloClNxM']+'_Training'+options['inTagIdent']

N = int(options['caloClNxM'].split('x')[0])
M = int(options['caloClNxM'].split('x')[1])

sparsityTag = str(options['sparsity']).split('.')[0]+'p'+str(options['sparsity']).split('.')[1]

**Only models with one input tensor are currently supported by VivadoAcceleratorBackend.
Therefore, the CNN model is in this case a special version that takes only the image as an input and gives as an output the flattened tensor out of the convolutions withou concatenating the position of the cluster.**

In [5]:
# quantized and pruned models
QCNN = keras.models.load_model(indir+'/CNNmodel_special4accelerator', compile=False)
QDNNident = keras.models.load_model(indir+'/ID_DNNmodel', compile=False)
QDNNcalib = keras.models.load_model(indir+'/CAL_DNNmodel', compile=False)

2023-07-04 14:09:17.315853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /grid_mnt/opt__exp_soft/llr/python/3.9.9/lib/python3.9/site-packages/cv2/../../lib64:/opt/exp_soft/llr/python/3.9.9/lib/vtk:/opt/exp_soft/llr/python/3.9.9/lib:/usr/lib64/classads:/usr/lib64:/usr/lib
2023-07-04 14:09:17.315897: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-07-04 14:09:17.316312: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) t

# QUANTIZED MODELS ONLY

In [6]:
############################## Pass quantized CNN model through hls4ml ##############################

compl = True
synth = False
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    QCNN_hls_cfg = hls4ml.utils.config_from_keras_model(QCNN, granularity='name')
    QCNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    QCNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in QCNN_hls_cfg['LayerName'].keys():
        QCNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Latency'
        QCNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        QCNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    QCNN_hls_cfg['LayerName']['TowerClusterImage']['Precision'] = 'ap_ufixed<10,8>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['ParallelizationFactor'] = 4
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<14,7>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<14,7>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['accum'] = 'ap_fixed<10,7>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer1']['Precision']['result'] = 'ap_fixed<10,7>'

    QCNN_hls_cfg['LayerName']['MP_CNNpBNlayer1']['Precision'] = 'ap_fixed<10,7>'

    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['weight'] = 'ap_fixed<5,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['bias'] = 'ap_fixed<6,1>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<14,6>'
    QCNN_hls_cfg['LayerName']['CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<14,6>'

    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['result'] = 'ap_fixed<9,6>'
    QCNN_hls_cfg['LayerName']['RELU_CNNpBNlayer2']['Precision']['accum'] = 'ap_fixed<9,6>'

    print(QCNN_hls_cfg)

    QCNN_hls = hls4ml.converters.convert_from_keras_model(QCNN,
                                                          hls_config=QCNN_hls_cfg,
                                                          output_dir=indir+'/CNNmodel_HLS_XCU200deployment/',
                                                          backend='VivadoAccelerator',
                                                          board='alveo-u200',
                                                          io_type='io_parallel',
                                                          platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QCNN_hls.compile()

if synth:
    QCNN_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=False)

Interpreting Model
Topology:
Layer name: TowerClusterImage, layer type: Input
Layer name: CNNpBNlayer1, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer1
Layer name: RELU_CNNpBNlayer1, layer type: QActivation
  -> Activation (quantized_relu(10,7)), layer name: RELU_CNNpBNlayer1
Layer name: MP_CNNpBNlayer1, layer type: MaxPooling2D
Layer name: CNNpBNlayer2, layer type: QConv2DBatchnorm
  -> Activation (linear), layer name: CNNpBNlayer2
Layer name: RELU_CNNpBNlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CNNpBNlayer2
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'TowerClusterImage': {'Precision': 'ap_ufixed<10,8>', 'Strategy': 'Latency', 'ReuseFactor': 1, 'Trace': True}, 'CNNpBNlayer1': {'Precision': {'weight': 'ap_fixed<5,1>', 'bias': 'ap_fixed<6,1>', 'accum': 'ap_fixed<14,7>', 'result': 'ap_fixed<14,7>'}, 'ReuseFactor': 1, 'Strategy': 'Latency', 'Trace': True, 'Pa

In [7]:
############################## Pass quantized identification DNN model through hls4ml ##############################

compl = True
synth = False
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    id_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNident, granularity='name')
    id_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    id_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in id_QDNN_hls_cfg['LayerName'].keys():
        id_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        id_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        id_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    id_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['accum'] = 'ap_fixed<13,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer1']['Precision']['result'] = 'ap_fixed<9,7>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer1']['Precision'] = 'ap_fixed<9,5>'

    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['accum'] = 'ap_fixed<11,6>'
    id_QDNN_hls_cfg['LayerName']['IDlayer2']['Precision']['result'] = 'ap_fixed<10,5>'
    id_QDNN_hls_cfg['LayerName']['RELU_IDlayer2']['Precision'] = 'ap_fixed<9,5>'

    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['weight'] = 'ap_fixed<2,1>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['accum'] = 'ap_fixed<7,4>'
    id_QDNN_hls_cfg['LayerName']['IDout']['Precision']['result'] = 'ap_fixed<7,4>'

    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Precision'] = 'ap_fixed<8,1>'
    id_QDNN_hls_cfg['LayerName']['sigmoid_IDout']['Strategy'] = 'Stable'

    print(id_QDNN_hls_cfg)

    QDNNident_hls = hls4ml.converters.convert_from_keras_model(QDNNident,
                                                               hls_config=id_QDNN_hls_cfg,
                                                               output_dir=indir+'/ID_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_parallel',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNident_hls.compile()

if synth:
    QDNNident_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=False)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: IDlayer1, layer type: QDense
  -> Activation (linear), layer name: IDlayer1
Layer name: RELU_IDlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_IDlayer1
Layer name: IDlayer2, layer type: QDense
  -> Activation (linear), layer name: IDlayer2
Layer name: RELU_IDlayer2, layer type: QActivation
  -> Activation (quantized_relu(8,5)), layer name: RELU_IDlayer2
Layer name: IDout, layer type: QDense
  -> Activation (linear), layer name: IDout
Layer name: sigmoid_IDout, layer type: Activation
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'IDlayer1': {'Precision': {'weight': 'ap_fixed<3,1>', 'accum': 'ap_fixed<13,6>', 'result': 'ap_fixed<9,7>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'IDlayer1_linear': {

In [8]:
############################## Pass quantized calibration DNN model through hls4ml ##############################

compl = True
synth = False
if compl:
    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=['Activation'],
                                                                                      rounding_mode='AP_RND',
                                                                                      saturation_mode='AP_SAT')

    # baseline model
    cal_QDNN_hls_cfg = hls4ml.utils.config_from_keras_model(QDNNcalib, granularity='name')
    cal_QDNN_hls_cfg['Model']['Precision'] = 'ap_fixed<16,6>'
    cal_QDNN_hls_cfg['Model']['ReuseFactor'] = 1
    for Layer in cal_QDNN_hls_cfg['LayerName'].keys():
        cal_QDNN_hls_cfg['LayerName'][Layer]['Strategy'] = 'Resources'
        cal_QDNN_hls_cfg['LayerName'][Layer]['ReuseFactor'] = 1
        cal_QDNN_hls_cfg['LayerName'][Layer]['Trace'] = True

    cal_QDNN_hls_cfg['LayerName']['middleMan']['Precision'] = 'ap_fixed<11,6>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['weight'] = 'ap_fixed<6,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['accum'] = 'ap_fixed<16,5>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer1']['Precision']['result'] = 'ap_fixed<16,7>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer1']['Precision'] = 'ap_fixed<11,7>'

    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['weight'] = 'ap_fixed<3,1>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['accum'] = 'ap_fixed<16,7>'
    cal_QDNN_hls_cfg['LayerName']['CALlayer2']['Precision']['result'] = 'ap_fixed<12,8>'
    cal_QDNN_hls_cfg['LayerName']['RELU_CALlayer2']['Precision'] = 'ap_fixed<10,7>'

    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['weight'] = 'ap_fixed<2,1>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['accum'] = 'ap_fixed<12,9>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Precision']['result'] = 'ap_fixed<10,9>'
    cal_QDNN_hls_cfg['LayerName']['CALout']['Strategy'] = 'Stable'

    print(cal_QDNN_hls_cfg)

    QDNNcalib_hls = hls4ml.converters.convert_from_keras_model(QDNNcalib,
                                                               hls_config=cal_QDNN_hls_cfg,
                                                               output_dir=indir+'/CAL_DNNmodel_HLS_XCU200deployment/',
                                                               backend='VivadoAccelerator',
                                                               board='alveo-u200',
                                                               io_type='io_parallel',
                                                               platform='xilinx_u200_xdma_201830_2')

    hls4ml.model.optimizer.get_optimizer('output_rounding_saturation_mode').configure(layers=[])
    QDNNcalib_hls.compile()

if synth:
    QDNNcalib_hls.build(csim=False, synth=True, vsynth=True, export=True, bitfile=False)

Interpreting Model
Topology:
Layer name: middleMan, layer type: Input
Layer name: CALlayer1, layer type: QDense
  -> Activation (linear), layer name: CALlayer1
Layer name: RELU_CALlayer1, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer1
Layer name: CALlayer2, layer type: QDense
  -> Activation (linear), layer name: CALlayer2
Layer name: RELU_CALlayer2, layer type: QActivation
  -> Activation (quantized_relu(9,6)), layer name: RELU_CALlayer2
Layer name: CALout, layer type: QDense
  -> Activation (linear), layer name: CALout
{'Model': {'Precision': 'ap_fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency'}, 'LayerName': {'middleMan': {'Precision': 'ap_fixed<11,6>', 'Strategy': 'Resources', 'ReuseFactor': 1, 'Trace': True}, 'CALlayer1': {'Precision': {'weight': 'ap_fixed<6,1>', 'accum': 'ap_fixed<16,5>', 'result': 'ap_fixed<16,7>'}, 'ReuseFactor': 1, 'Strategy': 'Resources', 'Trace': True}, 'CALlayer1_linear': {'Precision': 'ap_fixed<16,6>', 'Reuse

In [10]:
directory = '/data_CMS/cms/motta/Phase2L1T/2023_05_24_v15/TauMinator_CB_cltw5x9_Training/tensors_AlveoTest/'
X1 = np.load(directory+'/images_train.npz')['arr_0']

In [11]:
X1 = X1[:10]

In [15]:
hls_pred = QCNN_hls.predict(X1)

In [14]:
pynqbuffer = np.array([[5.25 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.625, 2.125,
             0.   , 0.   , 3.75 , 0.   , 0.   , 0.625, 2.5  , 2.   ,
             0.   , 0.   , 1.   , 0.   , 0.875, 1.25 , 0.   , 1.375],
            [5.375, 0.   , 0.   , 0.   , 0.   , 0.   , 0.625, 2.125,
             0.   , 0.   , 3.75 , 0.   , 0.   , 0.625, 2.625, 2.   ,
             0.   , 0.   , 1.   , 0.   , 0.75 , 1.25 , 0.   , 1.375],
            [1.125, 0.   , 0.   , 0.5  , 0.   , 0.   , 0.625, 0.   ,
             0.   , 0.25 , 0.875, 0.   , 0.   , 0.625, 0.75 , 0.   ,
             0.   , 0.25 , 0.375, 0.   , 0.5  , 0.625, 0.   , 0.   ],
            [1.375, 0.   , 0.   , 0.   , 0.   , 0.   , 0.5  , 0.   ,
             0.   , 0.125, 1.   , 0.   , 0.   , 0.5  , 0.25 , 0.125,
             0.   , 0.5  , 0.125, 0.125, 0.875, 0.75 , 0.   , 0.   ],
            [5.75 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 2.875,
             0.   , 0.   , 4.375, 0.   , 0.   , 0.5  , 2.625, 2.5  ,
             0.   , 0.   , 1.375, 0.   , 0.875, 1.375, 0.   , 1.875],
            [5.75 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.125, 2.625,
             0.   , 0.   , 4.375, 0.   , 0.   , 0.375, 2.875, 2.625,
             0.   , 0.   , 1.625, 0.   , 0.   , 1.75 , 0.   , 2.   ],
            [1.375, 0.   , 0.   , 0.   , 0.   , 0.   , 0.625, 0.   ,
             0.125, 0.   , 1.5  , 0.   , 0.   , 1.   , 0.25 , 0.5  ,
             0.   , 0.125, 0.5  , 0.   , 0.5  , 0.625, 0.   , 0.375],
            [1.375, 0.   , 0.   , 0.   , 0.   , 0.125, 0.625, 0.   ,
             0.   , 0.   , 1.375, 0.   , 0.   , 0.875, 0.   , 0.5  ,
             0.   , 0.375, 0.25 , 0.   , 0.75 , 0.75 , 0.   , 0.   ],
            [0.625, 0.   , 0.   , 0.25 , 0.   , 0.   , 0.375, 0.   ,
             0.   , 0.375, 0.625, 0.25 , 0.   , 0.625, 0.75 , 0.   ,
             0.   , 0.5  , 0.125, 0.25 , 0.625, 0.5  , 0.   , 0.   ],
            [0.75 , 0.   , 0.   , 0.375, 0.25 , 0.   , 0.5  , 0.   ,
             0.   , 0.25 , 0.5  , 0.25 , 0.   , 0.375, 0.5  , 0.   ,
             0.   , 0.375, 0.125, 0.   , 0.375, 0.5  , 0.   , 0.   ]])

In [16]:
hls_pred - pynqbuffer

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 